In [1]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile
from tqdm import tqdm

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [2]:
serval_data_folder = "../data"

In [3]:
input_wav_samples_folder                     = serval_data_folder + "/wav_samples_custom"
wav_samples_custom_labels_csv                = input_wav_samples_folder + "/wav_samples_custom_labels.csv"
target_wav_samples_custom_all_enumerated_csv = serval_data_folder + "/wav_samples_custom/wav_samples_custom_all_enumerated_and_labeled.csv"

In [4]:
# read class labels
df_class_labels = pd.read_csv(wav_samples_custom_labels_csv, sep=";")
display(df_class_labels)

,label,mid,folder_name,description,enabled,source
0,1000,/c/a_1000,0_silence,Silence,1,custom_amsterdam_sample
1,1001,/c/a_1001,556_gunshots,556 gunshots,1,custom_amsterdam_sample
2,1002,/c/a_1002,a_brommer_alarms,Brommer Alarm,1,custom_amsterdam_sample
3,1003,/c/a_1003,b_brommers,Brommer,1,custom_amsterdam_sample
4,1004,/c/a_1004,c_claxons,Claxons,1,custom_amsterdam_sample
5,1005,/c/a_1005,k_dichtklappende_autodeuren,Dichtklappende autodeuren,1,custom_amsterdam_sample
6,1006,/c/a_1006,m_luide_mensen_schreeuwen_en_zo,"Luide mensen (schreeuwen, enz)",1,custom_amsterdam_sample
7,1007,/c/a_1007,o_motoren,Motoren,1,custom_amsterdam_sample
8,1008,/c/a_1008,t_terrasgeluiden,Terrasgeluide (stille mensen),1,custom_amsterdam_sample
9,1009,/c/a_1009,v_versterkte_muziek,Versterkte muziek,1,custom_amsterdam_sample


In [5]:
# Enumerate all the available wav files, label them and store a data-frame of the result
def collectAndLabelSamples(full_path_name, label, mid, source, display_name):
    ## Read and rewrite all test files
    files = gfile.Glob(str(full_path_name + "/*.wav"))
    
    df = pd.DataFrame(columns=['label', 'mid', 'display_name', 'source', 'filename', 'filepath'])
    for file in tqdm(files):
        df_row = pd.DataFrame({'label':[label], 
                               'mid':[mid], 
                               'display_name':[display_name], 
                               'source':source, 
                               'filename':[ntpath.basename(file)],
                               'filepath':[file]})
        df = df.append(df_row, ignore_index = True)
    return(df)

In [6]:
## Enumerate all custom wav-files and label them
df_all_wav_samples = pd.DataFrame(columns=['label', 'mid', 'display_name', 'source', 'filename', 'filepath'])
for index, row in df_class_labels.iterrows():
    if row['enabled'] == 1:
        print('Processing record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')
        df_row = collectAndLabelSamples(input_wav_samples_folder + '/' + row.folder_name, 
                                        row.label,
                                        row.mid,
                                        row.source,
                                        row.description)
        df_all_wav_samples = df_all_wav_samples.append(df_row, ignore_index = True)
    else:
        print('Skipping record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')

 10%|█         | 90/876 [00:00<00:00, 892.93it/s]

Processing record 0; label=1000, description=Silence, path=0_silence
Processing record 1; label=1001, description=556 gunshots, path=556_gunshots


 11%|█         | 89/812 [00:00<00:00, 881.81it/s]

Processing record 2; label=1002, description=Brommer Alarm, path=a_brommer_alarms
Processing record 3; label=1003, description=Brommer, path=b_brommers


  0%|          | 0/164 [00:00<?, ?it/s]

...

Processing record 6; label=1006, description=Luide mensen (schreeuwen, enz), path=m_luide_mensen_schreeuwen_en_zo


 16%|█▌        | 85/536 [00:00<00:00, 842.56it/s]

Processing record 7; label=1007, description=Motoren, path=o_motoren
Processing record 8; label=1008, description=Terrasgeluide (stille mensen), path=t_terrasgeluiden


100%|██████████| 34/34 [00:00<00:00, 909.73it/s]

Processing record 9; label=1009, description=Versterkte muziek, path=v_versterkte_muziek


In [7]:
## Write df_all_wav_samples to csv
df_all_wav_samples.to_csv(target_wav_samples_custom_all_enumerated_csv, sep=';')

In [8]:
display(df_all_wav_samples.head())
display(df_all_wav_samples.groupby(['label', 'mid', 'display_name']).agg(['nunique']))
display(df_all_wav_samples.agg(['nunique']))

,label,mid,display_name,source,filename,filepath
0,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_02 (copy 3).wav,../data/wav_samples_custom/0_silence/silence_0...
1,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_05 (copy 4).wav,../data/wav_samples_custom/0_silence/silence_0...
2,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_03 (copy 4).wav,../data/wav_samples_custom/0_silence/silence_0...
3,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_05 (copy 5).wav,../data/wav_samples_custom/0_silence/silence_0...
4,1000,/c/a_1000,Silence,custom_amsterdam_sample,silence_04 (copy 5).wav,../data/wav_samples_custom/0_silence/silence_0...


,,,source,filename,filepath
,,,nunique,nunique,nunique
label,mid,display_name,,,
1000,/c/a_1000,Silence,1,30,30
1001,/c/a_1001,556 gunshots,1,876,876
1002,/c/a_1002,Brommer Alarm,1,23,23
1003,/c/a_1003,Brommer,1,812,812
1004,/c/a_1004,Claxons,1,132,132
1005,/c/a_1005,Dichtklappende autodeuren,1,164,164
1006,/c/a_1006,"Luide mensen (schreeuwen, enz)",1,634,634
1007,/c/a_1007,Motoren,1,51,51


,label,mid,display_name,source,filename,filepath
nunique,10,10,10,1,3292,3292
